In [ ]:
import pyvisgraph as vg
import math
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Polygon, Point, LineString
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import CubicSpline, make_interp_spline, make_smoothing_spline, splrep
import thymio
import controller as control
import time
from IPython.display import clear_output


#Define the obstacles
O1 = [(3,8),(5,5),(8,7),(6,10)]
O2 = [(20,5),(23,7),(28,7),(24,2)]
O3 = [(18,9),(16,12),(21,15),(23,11)]
O4 = [(32,12),(34,15),(40,14),(37,11)]
obstacles = [O1,O2,O3,O4]

#Create the polygons as objects and plot them using Geoseries
pol= []
for ob in obstacles:
    pol.append(Polygon(ob))
env = GeoSeries(pol)
GeoSeries.plot(env)

#Enlarge the polygons and plot them again (bla)
margin = 1
env_dilated = env.buffer(margin,join_style = 2)
GeoSeries.plot(env_dilated)



: 

In [ ]:
#Convert the dilated polygons in a format suitable for visibility graph  -> VS points
pol_for_vg = []
for poly in env_dilated:
        x, y = poly.exterior.coords.xy #gets coordinates of the vertices 
        polygon_vg = []
        for i in range(len(x)):
            polygon_vg.append(vg.Point(x[i],y[i]))
        pol_for_vg.append(polygon_vg)

#Create the visibility graph
graph = vg.VisGraph()
graph.build(pol_for_vg)

: 

In [ ]:
#Define start and end and convertem to format suitable for the vg.graph --> MAYBE IT COULD BE USEFUL TO IMPLEMENT A FUNCTION DOING THESE CONVERSIONS

start = vg.Point(1,15)
end = vg.Point(37,2)

#Find the shortest path-> gives back the set of corner through which one must pass
shortest_path  = graph.shortest_path(start,end)

print(shortest_path)




: 

In [ ]:
#Convert the path found in a list of points suitable for geoseries
path_gs = []
path_tuples = []
for el in shortest_path:
    path_gs.append(Point(el.x,el.y))
    path_tuples.append((el.x,el.y))
path_tuples = np.array(path_tuples)



#Get the dilated polygons into a list recreate the environent
polygon_dilated_for_gs = []
for poly in env_dilated:
    polygon_dilated_for_gs.append(poly)

#Add the path to the new geometry and create the environemnt
polygon_dilated_for_gs.append(LineString(path_gs))
unsmoothed_path = GeoSeries(LineString(path_gs))
geom_fin = polygon_dilated_for_gs
env_final = GeoSeries(geom_fin)

#Plot the final environment
GeoSeries.plot(env_final)


: 

In [ ]:
#Calculate the distance: since they are straight line, the path lenght can be found through Pytagorean theorem between the points identified
distance = 0
for i in range(len(path_gs)-1):
    distance += np.sqrt((path_gs[i].x-path_gs[i+1].x)**2+(path_gs[i].y-path_gs[i+1].y)**2)

print(f"The shortest path has length {distance:.2f}")



: 

In [ ]:
#Smoothen the path

# Generate a finer set of points for a smoother curve
x_points = np.array(path_tuples[:,0])
x_finer = np.linspace(x_points[0], x_points[-1], 30)  # Adjust the number of points as needed


# Create a cubic spline interpolation on the finer set of points
cs = CubicSpline(path_tuples[:,0], path_tuples[:,1])
spline_interp = make_interp_spline(path_tuples[:,0], path_tuples[:,1])
spline_smooth = make_smoothing_spline(path_tuples[:,0], path_tuples[:,1]) # We are going to use this one because it is closer to the optimal path

# Obtain the corresponding y-coordinates on the smoothed curve
y_finer_cubic = cs(x_finer)
y_finer_interp = spline_interp(x_finer)
y_finer_smooth = spline_smooth(x_finer)
path = np.matrix.transpose(np.array([x_finer,y_finer_smooth]))
print(np.shape(path))
print(path)

smooth_path = []
#get back to gs format
smooth_path = gpd.points_from_xy(x_finer, y_finer_cubic)


new_path_gs = GeoSeries(smooth_path)
ax1 = GeoSeries.plot(new_path_gs)
ax2 = GeoSeries.plot(env_dilated,color='red')
GeoSeries.plot(env,color='blue',ax=ax2)
GeoSeries.plot(unsmoothed_path,color='black',ax=ax2)
#plt.plot(x_finer,y_finer_cubic,color='green')
plt.plot(x_finer,y_finer_interp,1,color='purple')
plt.plot(x_finer,y_finer_smooth,color='orange')
plt.show()

: 

In [ ]:
# Start asynchronous communication with Thymio
from tdmclient import ClientAsync, aw
client = ClientAsync()            # Create client object
client.process_waiting_messages() # Ensure connection to TDM (Thymio Device Manager)
node = aw(client.wait_for_node()) # Ensure connection with Thymio robot (node)
aw(node.lock())                   # Lock robot to change variables and run programs

: 

In [ ]:
# # Check if motors are working
# thymioMaxSpeed = 500
# thymio.set_motor_speed(50,50,node)
# aw(client.sleep(1.0))
# thymio.stop_motor(node)
# thymio.set_motor_speed(thymioMaxSpeed,thymioMaxSpeed,node)
# motorLeftSpeed = 0
# motorRightSpeed = 0
# while motorLeftSpeed != thymioMaxSpeed and motorRightSpeed != thymioMaxSpeed:
#     [motorLeftSpeed, motorRightSpeed] = thymio.get_motor_speed(node,client)
#     print("\nMotor Left Speed: ",flush=True)
#     print(motorLeftSpeed,flush=True)
#     print("\nMotor Right Speed: ",flush=True)
#     print(motorRightSpeed,flush=True)
#     clear_output(wait=True)
    
# thymio.stop_motor(node)

: 

In [ ]:
# thymio.stop_motor(node)

: 

In [ ]:
# # Check Proximity Sensors
# prox = thymio.get_distances(node,client)
# print(prox)

: 

In [ ]:
# # Unlock the Robot (ALWAYS RUN AFTER RUNNING PROGRAM)
# aw(node.unlock())

: 

In [ ]:
#PART 1: Create potential field map from Geoseries environment
from localnavigation import *

#Define the size of the environement
x_size = 45
y_size = 20

pf_map = potential_field_map(env_dilated,path[-1],(x_size,y_size), k_att=0.1, k_rep=1.0, rep_radius=5.0)

: 

In [ ]:
#Plot the potential field obtained

plot_potential_map_arrows_on_geoseries(pf_map, env_dilated, path[-1])

plot_potential_map_arrows(pf_map, path[-1])

: 

In [ ]:
#PART 2: Create a temporary potential field to add local obstacles

#Data for trial, most of them will be coming from the robot/other parts
robot_position = ( 14,15)
prox_sensor_readings = [ 3000, 2500, 2000, 1110, 500, 7]
robot_angle = 30
robot_radius = 3
delta_angle = 20 #delta angle of the sensors considering all together right and left ones
map_size = (45,20)
tol = 2000
scale_factor = 1000 #to be found and scaled to the map dimension
k_rep = 1

temporary_map = potential_field_temp(robot_position, prox_sensor_readings, robot_radius, robot_angle, delta_angle,map_size,tol,scale_factor,k_rep=1)

plot_potential_map_arrows(temporary_map, robot_position)

: 

In [ ]:
#PART 3: Sum temporary potential map and global map, to find the overall potential

overall_map = pf_map + temporary_map

plot_potential_map_arrows(overall_map, path[-1])

: 

In [ ]:
 #PART 4: Using the map compute the next position to be in to be fed to the controller

epsilon = 1
step_size = 0.5 #to be chosen

next_point= find_next_position(robot_position,step_size,epsilon,overall_map)

print(next_point)

: 

In [ ]:
#Repeat the global navigation tasks performed above by using some functions which will work with any data (hopefully)
from globalnavigation import *

#PART 1: CREATE THE ENVIRONMENT
#Define the obstacles
O1 = [(3,8),(5,5),(8,7),(6,10)]
O2 = [(20,5),(23,7),(28,7),(24,2)]
O3 = [(18,9),(16,12),(21,15),(23,11)]
O4 = [(32,12),(34,15),(40,14),(37,11)]
obstacles = [O1,O2,O3,O4]

#Create the polygons
polygons = create_polygons(obstacles)

#Create and plot geoseries environment
environment = GeoSeries(polygons)
GeoSeries.plot(environment)

#Dilate the obstacles
robot_size = 1
environment_dil = environment.buffer(robot_size/2, join_style = 2)
GeoSeries.plot(environment_dil, color ='green')

: 

In [ ]:
 #PART 2: CREATE THE VISIBILITY GRAPH

#Extract the corners of the dilated environment
corners = extract_coordinates(environment_dil)

#Convert them to points in visibility graph format
corners_vg = convert_pol_to_vs(corners)

#Build the graph
graph = vg.VisGraph()
graph.build(corners_vg)

: 

In [ ]:
#PART 3: COMPUTE SHORTEST PATH

#Define start and end and convertem to format suitable for the vg.graph 
start_position = (1,15)
end_position = (38,3)

start = vg.Point(start_position[0], start_position[1])
end = vg.Point(end_position[0], end_position[1])

#Find the shortest path-> gives back the set of corner through which one must pass
shortest_path  = graph.shortest_path(start,end)

print(shortest_path)

#Convert the path found in shpaely points ( for Geoseris) and tuples (for interpolation)
sp_gs, sp_tuples =convert_path_vs(shortest_path)

#Recreate the environment for plotting: dilated polygons + path
pol_dil = create_polygons(corners)
pol_dil.append(LineString(sp_gs))
geom_complete = pol_dil
env_complete = GeoSeries(geom_complete)
GeoSeries.plot(env_complete)

: 

In [ ]:
#PART 4: SMOOTHEN THE PATH

#Get a finer set of points for the path
smooth_path_tuples = smoothen_path(np.array(sp_tuples))

#Get back to gs format
x = smooth_path_tuples[:,0]
y = smooth_path_tuples[:,1]
smooth_path = gpd.points_from_xy(x, y)


new_path_gs = GeoSeries(smooth_path)
ax1 = GeoSeries.plot(new_path_gs)
ax2 = GeoSeries.plot(env_dilated,color='red')
GeoSeries.plot(env,color='blue',ax=ax2)
GeoSeries.plot(unsmoothed_path,color='black',ax=ax2)
plt.plot(x_finer,y_finer_interp,1,color='purple')
plt.show()

: 

In [ ]:
# Control Loop Flags
isFirstLoop = True
isAtEndGoal = False
isBlocked = False

# Robot Path Variables
path = path                                            # TODO: Path as array of waypoints (XY tuples)
current_position = path[0]                             # TODO: Initial Location defined as current position of Thymio
end_goal = path[-1]                                    # TODO: End goal is final position of planned path

# Robot Pose Variables
initial_orientation = 0.0                              # TODO: Orientation from Localization
current_pose = [current_position, initial_orientation] # Define the current robot pose [x y theta]
initial_pose = current_pose                            # Save initial pose of robot to robot object
current_orientation = initial_orientation              # Current orientation theta of the robot             

# PD Controller Gains
kp = 0.5  # Proportional gain
kd = 0.2  # Derivative gain

# Looping Variable 
target_index = 1  # Index of next targeted position on path

# Create Robot and Controller Objects
Robot = thymio.Robot(node,client,initial_pose,current_pose)
PD = control.PDController(kp,kd)

while target_index < len(path):
    # Increment along path of tuples
    target_point = path[target_index]

    # Check if there is an obstacle in front of us (if so, enter local navigation loop)
    if Robot.isBlocked():
        isBlocked = True
        while isBlocked:
            # Stop the Robot from moving
            Robot.stop_motor()

            # Create Potential Field Map from current known obstacles
            pf_map = Navigation.create_potential_field_map(geo_env, goal_pos,map_size)

            # Add obstacle to Potential Field from Distance sensor values
            temporary_map = Navigation.potential_field_temp(robot_position, prox_sensor_readings, robot_radius, robot_angle, delta_angle,map_size,tol,scale_factor,k_rep=1)

            overall_map = pf_map + temporary_map

            Navigation.plot_potential_map_arrows(overall_map, path[-1])

            epsilon = 1
            step_size = 0.5 #to be chosen

            next_point= Navigation.find_next_position(robot_position,step_size,epsilon,overall_map)

            

    # Calculate steering angle
    Robot.calculate_steering_angle(target_point)

    # Calculate distance error (for PD control)
    PD.calc_cross_track_error(Robot.current_pose,path)

    # PD control to correct current steering angle
    correction = PD.pd_controller()

    # Apply correction to steering angle
    Robot.current_steering_angle += correction

    # Convert steering angle to Thymio wheel speeds
    left_speed, right_speed = Robot.convert_to_wheel_speeds(Robot.current_steering_angle, Robot.max_speed, Robot.wheelbase)

    # Apply the speeds to the robot's wheels (replace this with your actual code)
    Robot.set_motor_speed(left_speed,right_speed)

    # Update current position (TODO: Replace this with actual vision or localization with Robot.update() )
    Robot.current_pose[0:2] = path[target_index-1] 
    Robot.current_orientation += 0.1*2*np.pi # [TODO] Get Orienation

    # Get current distance b/w robot and target point
    distance_to_target = PD.distance_to_target(Robot.current_pose[0:2],target_point)

    # Check if the robot reached the target point within a threshold
    if PD.isAtTargetPoint(distance_to_target):
        target_index += 1  # Move to the next target point

    time.sleep(0.1)  # Time interval for control loop

print("Reached end of trajectory.")

: 

In [ ]:
# # Constants
# wheelbase = 0.112  # Distance between wheels in meters

# # Control Loop Flags
# isFirstLoop = True
# isAtTarget = False
# isAtEndGoal = False

# # Initialize Robot Position and Waypoints
# path = path                                              # Path as array of waypoints (XY tuples)
# current_position = path[0]                                # Initial Location defined as current position of Thymio
# endGoal = path[-1]                                       # End goal is final position of planned path
# initialOrientation = 0.0                                 # TODO: Orientation from Localization
# robotCurrentPose = [current_position, initialOrientation] # Define the current robot pose [x y theta]
# robotMaxSpeed = 100                                       # Max speed of the robot (set to target speeds)
# orientation = initialOrientation

# # PD Controller Gains
# kp = 0.5  # Proportional gain
# kd = 0.2  # Derivative gain

# # PD Controller Variables
# last_error = 0

# # Looping Variable
# target_index = 1

# while target_index < len(path):
#     target_point = path[target_index]

#     # Calculate steering angle
#     steering_angle = thymio.calculate_steering_angle(current_position, target_point,orientation)

#     # Calculate distance error (for P control) as norm of 2 points
#     distance_error = math.sqrt((target_point[0] - current_position[0])**2 + (target_point[1] - current_position[1])**2)

#     # PD control to adjust steering
#     correction = control.pd_controller(distance_error,last_error,kp,kd)

#     # Save Distance Error (For D control)
#     last_error = distance_error

#     # Apply correction to steering angle
#     steering_angle += correction

#     # Convert steering angle to Thymio wheel speeds
#     left_speed, right_speed = thymio.convert_to_wheel_speeds(steering_angle, robotMaxSpeed, wheelbase)

#     # Apply the speeds to the robot's wheels (replace this with your actual code)
#     thymio.set_motor_speed(left_speed,right_speed,node)
#     print("Left Wheel Speed:", left_speed)
#     print("Right Wheel Speed:", right_speed)
#     time.sleep(0.1)  # Time interval for control loop

#     # Update current position (replace this with actual odometry or sensor data)
#     current_position = path[target_index-1]

#     # Check if the robot reached the target point within a threshold
#     if distance_error < 0.1:
#         target_index += 1  # Move to the next target point

#     orientation += 0.1*2*np.pi # [TODO] Get Orienation

# print("Reached end of trajectory.")

: 